# Create a local information system of a Thin-Section

Author: @gianfrancodp - CC-BY-SA


---

Table of Content

1. [Import files](#1-import-files)
2. [Create Raster tiles using Gdal](#2-create-raster-tiles-using-gdal)
3. [Convert SHP to Geojson using GDAL ogr2ogr](#3-convert-shp-to-geojson-using-gdal-ogr2ogr)
4. [Add GeoJson overlay to web-viewer](#4-add-geojson-overlay-to-web-viewer)
5. [Add pop-up feature to the map](#5-add-popup-to-the-map)
6. [Add rosediagram and legend feature to the webpage](#6-add-mineral-legend-and-rosediagrams)

---

### Python and enviromnent main requirements:

- Python 3
- GDAL
- beautifulsoup4==4.12.3
- ipython==8.12.3
- ipywidgets==8.1.2

### Credits and library used:

- ["OpenLayers" Library](https://github.com/openlayers/openlayers)
- "OpenLayer Extension" AKA `ol-ext` ([source code](https://github.com/Viglino/ol-ext?tab=readme-ov-file)) released under BSD 3-Clause License. Copyright (c) 2016-2021, Jean-Marc Viglino, IGN-France [All rights reserved](https://github.com/Viglino/ol-ext?tab=BSD-3-Clause-2-ov-file)
- [GDAL](https://gdal.org/en/latest/index.html) and [gdal2tiles](https://gdal.org/en/latest/programs/gdal2tiles.html) © 1998-2024 Frank Warmerdam, Even Rouault, and others
- [Beautiful Soap](https://www.crummy.com/software/BeautifulSoup/) 1996-2024 Leonard Richardson. Unless otherwise noted, Creative Commons License.
- [GeoJSON](https://geojson.org/) standard definition [RFC7946](https://datatracker.ietf.org/doc/html/rfc7946) (Butler et al, 2016)
- [Micro-Fabric Analyzer (MFA)](https://www.mdpi.com/2220-9964/10/2/51) (Visalli et al, 2021).
- [Quantitative X-ray Map Analyser (Q-XRMA)](https://www.sciencedirect.com/science/article/pii/S0098300417306982) (Ortolano et al, 2018)
- ["ArcStereoNet](https://doi.org/10.3390/ijgi10020050): A New ArcGIS® Toolbox for Projection and Analysis of Meso- and Micro-Structural Data" (Ortolano et al, 2021)


 
In this section we collect input data and global settings.
We assume that we have a raster file of a Thin-section digitalized in any way and we have defined a polygon feature that overlay minerals or other elements of the thin-section.

The poligon feature layer must be in a GIS-readable format.
We use gadl2tiles with no-crs definition

In [1]:
# verify Python version
import sys
if sys.version_info[0] < 3:
    raise Exception("Python 3 not detected.")
else :
    print("Python 3 detected. OK.")

# verify GDAL version
try:
    from osgeo import gdal
    print(f"GDAL version: {gdal.__version__}")
except ImportError:
    raise ImportError("GDAL not detected.")

Python 3 detected. OK.
GDAL version: 3.8.5


### How to install GDAL binary
If your system do not have GDAL installed before continue you must to install it.

as explained in [offical repository](https://gdal.org/en/latest/download.html#binaries) to install GDAL in your system follow these steps:

- **Windows** builds are available via [Conda Forge](https://anaconda.org/conda-forge/gdal) (64-bit only). GDAL is also distributed by [GISInternals](https://www.gisinternals.com/index.html) and [OSGeo4W](https://trac.osgeo.org/osgeo4w/) and through the [NuGet](https://www.nuget.org/packages?q=GDAL) and [vcpkg](https://gdal.org/en/latest/download.html#vcpkg) package managers. *For windows users we suggest use Conda, read more on [official web page](https://gdal.org/en/latest/download.html#conda) for installation process.*

- **Linux** packages are available for [Debian](https://tracker.debian.org/pkg/gdal), Alpine_, Fedora_, and other distributions.
- In **Mac OS** GDAL packages are available on [Homebrew](https://formulae.brew.sh/formula/gdal). If you have homebrew installed just type `homebrew install gdal` in the terminal.

# 1. Import files

In this section we use `upload_files_widgets` from `lis_function.py` to import data and files usinng widgets.

Next, store the file in a local subfolder with the same name of "project name" gived by the user.

## 1.1 File specification details

1. <u>**Shapefile**</u>: the minimum collection of 4 files [.shp, .dbf, .shx, .cpg] in ESRI standard format. the shapefile will be converted in GeoJSON format by `ogr2ogr` subprocess. The Shapefile must cointain polygons of mineral of the thin section. The element that overlay the raster image.

**Field table definition used in this LIS and stored in Shapefile**
|Field name|type|example value|NOTE|
|---|---|---|---|
|*Mineral*|str|`Pl`|Coded name of mineral|
|O|float|`90.0`|Degree of orientation|
|Asr|float|`0.39438`|Aspect Ratio|
|A|float|`0.12928`|Area in micrometers|
|R|float|`0.58411`|Roundness|
|GSI|float|`1.87833`|Grain Shape Index|

The code of minerals to use is reported in the following table:

### Codename of mineral used
|*Code*|Mineral name|
|---|---|
|Amph|Amphibole|
|Ep|Epidote|
|Ap|Apatite|
|Kfs|K-Feldspar|
|Ol|Olivine|
|Pl|Plagioclase|
|Px|Pyroxene|
|Qtz|Quartz|
<!-- |Cal|Calcite|
|Ca-Si min|Calc-Silicate Mineral|
|Scap (aggr)|Scapolite (aggregate)|
|Scap|Scapolite|
|Oth|Other| -->


In this example polygons are generated by these tools:
- [Micro-Fabric Analyzer (MFA)](https://www.mdpi.com/2220-9964/10/2/51) (Visalli et al, 2021).
- [Quantitative X-ray Map Analyser (Q-XRMA)](https://www.sciencedirect.com/science/article/pii/S0098300417306982) (Ortolano et al, 2018)



2. <u>**Image**</u>: The original file raster used for webview, will be created hundreds of raster-tiles, there are in a format readable by `gdal` and browser, we suggest Jpeg or TIFF format

3. <u>**NO-Coordinate Reference System (NO-CRS)**</u>: check that both Shapefile and Image are correctly overlapped in a NO-CRS system. We only use the internal coordinate of the pixel of the raster, polygon vertex defined in Shapefile are referring to a specific pixel of the Imagefile.


In [7]:
# Input widgets

from lis_functions import upload_files_widgets
Shapefile_selector, Jpgfile_selector, projectname = upload_files_widgets()
# Shapefile_selector, Jpgfile_selector, projectname, EPSGcode = upload_files_widgets()


Label(value='Select ALL the shapefile files [.shp, .dbf, .shx, .cpg]')

FileUpload(value=(), accept='.shp, .dbf, .shx, .cpg', description='Shapefile polygon layer', multiple=True)

Label(value='Select the Image file')

FileUpload(value=(), description='Image')

Label(value='Select the project name')

Text(value='', description='Name:', placeholder='NAME')

Label(value='Note: ouptut folder will be created in the same directory as the input files')

In the windows above select file in the "example_data" subfolder 
![example](https://i.imgur.com/lDIHotO.png)

In [8]:
from lis_functions import save_to_temp_dir

# Save file to a temporary directory

files_path = save_to_temp_dir(Shapefile_selector, Jpgfile_selector, projectname.value)

# Run gdal2tiles


# 2. Create Raster tiles using Gdal

Using a subprocess, we run the function `run_gdal2tiles`, that launch the gdal2tiles procedure in terminal.

By default a simple webviewer is written and stored in openalayer.html file inside the project folder.

NOTE: destination folder is also created by `save_to_temp_dir` function before

In [9]:
from lis_functions import run_gdal2tiles

# Create raster tiles using gdal2tiles
# for customization see function in lis_functions.py

run_gdal2tiles(files_path['Raster_path'], projectname.value)

# some information is stored in tilemapsoruce.xml file generated from gdal2tiles

WARNING! - Directory already exists, Files with same name will be overwrited.


Generating Base Tiles:


0...10...20...30...40...50...60...70...80...90...100 - done.
0.

Generating Overview Tiles:


..10...20...30...40...50...60...70...80...90...100 - done.
Tiles stored in: PAL22


# 3. Convert SHP to Geojson using GDAL ogr2ogr

Using `ogr2ogr` in a subprocess we convert shp in geoJson format


In [10]:
import os

from lis_functions import convert_shp_to_geojson

# Get the input shapefile path from `save_to_temp_dir` cell
input_shp = files_path['Shp_file_path']

# Create the output GeoJSON file path
output_geojson = projectname.value + '/' + os.path.basename(input_shp) +'.geojson'

# Convert SHP to GeoJSON using the function and an ogr2ogr subprocess
convert_shp_to_geojson(input_shp, output_geojson)

Conversion completed: PAL22/PAL22_min_porphy.shp.geojson


# 4. Add GeoJson overlay to web-viewer

We access the openalyers.html defautl file generated by gdal2tiles and add some simple javascript code to overlay the previously created GeoJSON.

Using BeautifulSoup Python library the function add script tags for overlaying a GeoJSON inside the OpenLayers Javascript system.

**NOTE**: For a properly map overlay viweing in a browser, we assume a nominal reference system as EPSG:3857, the same as  used by OpenLayers by default. If other SRSs are used, the vector overlay on raster tiles is not correct.

This is a fictitious hypothesis only for the execution of the webviewer in HTML code. Like an hack that exploits the OpenLayer libraries, which are instead designed for maps with well-defined reference systems. It only works if the vector layer and raster layer are already overlaid (according to the numerical values of their respective coordinates), and have been saved without SRS definition.

In [11]:
from lis_functions import add_geojson_overlay_to_gdal2tiles_html_output

# Note: the default output of gdal2tiles is an html file named 'openlayers.html'
# in the output directory. This file is a template and can be modified to include

html_input = projectname.value + '/openlayers.html'

html_output = projectname.value + '/index.html'

geojson_file_path = output_geojson

add_geojson_overlay_to_gdal2tiles_html_output(geojson_file_path, html_input, html_output)


File updated: PAL22/index.html


# 5. Add popup to the map

We now embed the library "OpenLayer Extension" AKA `ol-ext` see [source code](https://github.com/Viglino/ol-ext?tab=readme-ov-file) released under BSD 3-Clause License. Copyright (c) 2016-2021, Jean-Marc Viglino, IGN-France [All rights reserved](https://github.com/Viglino/ol-ext?tab=BSD-3-Clause-2-ov-file) in html file.

the function called `add_popup_feature_to_gdal2tiles_html_output(Html_input,Html_output)` work on html file.
Steps are:
1. Add link to JS and CSS of `ol-ext` hosted by cdn.jsdelivr.net in the head and bottom of html
2. Add a precise customized script with `ol.Overlay.PopupFeature()`
3. Add customized CSS overrides for better visualization.

The customized popup work with precise field name in the geojson file used, in this example we have a template like this:


```javascript
template: {
        title: 
          function(f) {
            return f.get('Mineral') + ' - ' + getMineralName(f.get('Mineral'));
          },
        attributes: //
            {
            'O' : { 
                    title: 'Orientation',
                    before:'', 
                    format: ol.Overlay.PopupFeature.localString(),
                    after:'°' 
                },
            'AsR' : { title: 'Aspect Ratio' },
            'R': { title: 'Roundness' },
            'A': { title: 'Area' },
            'GSI': {title: 'Grain Shape Index'} // ,
            }
        }
```
Mineral names in the the popup title is generating with the ["Codename of mineral used table"](#codename-of-mineral-used):

<!-- |Code (str)| Mineral name|
|---|---|
|Amph|Amphibole|
|Ep|Epidote|
|Ap|Apatite|
|Kfs|K-Feldspar|
|Ol|Olivine|
|Pl|Plagioclase|
|Px|Pyroxene|
|Qtz|Quartz|
|Cal|Calcite|
|Ca-Si min|Calc-Silicate Mineral|
|Scap (aggr)|Scapolite (aggregate)|
|Scap|Scapolite|
|Oth|Other| -->


Change this in the function definition if are different on your field table.

In [12]:

from lis_functions import add_popup_feature_to_gdal2tiles_html_output

add_popup_feature_to_gdal2tiles_html_output('PAL22/index.html', 'PAL22/index2.html')

File updated: PAL22/index2.html


# 6. Add mineral legend and rosediagrams

Now we add to html another customization.
With *ArcStereoNet* [*] or other software for analizing microstructural data of Thin section, we obtain weighted and unweighted rose diagram based on grains cumulative area.

[*] Ortolano, Gaetano, Alberto D’Agostino, Mario Pagano, Roberto Visalli, Michele Zucali, Eugenio Fazio, Ian Alsop, and Rosolino Cirrincione. 2021. "ArcStereoNet: A New ArcGIS® Toolbox for Projection and Analysis of Meso- and Micro-Structural Data" ISPRS International Journal of Geo-Information 10, no. 2: 50. https://doi.org/10.3390/ijgi10020050

In the folder `templates\asset\` are stored SVGs files of rose diagrams and customized legend SVG (using the same color)

In the HTML now we ad at the bottom:
- a legend icon buttons for each mineral detected inside the Thin Section considered
- a blank rose diagram that be updated after the event of click on a specific mineral grain.
- a JS code for the features


In [1]:
# from bs4 import BeautifulSoup
# import os
from lis_functions import add_legend_and_rosediagrams


# create a dictionary of legend icons paths
legend_icons = {
    'Ap': 'example_data/asset/legend/Ap_legend.svg',
    'Amph': 'example_data/asset/legend/Amph_legend.svg',
    'Ep': 'example_data/asset/legend/Ep_legend.svg',
    'Kfs': 'example_data/asset/legend/Kfs_legend.svg',
    'Ol': 'example_data/asset/legend/Ol_legend.svg',
    'Pl': 'example_data/asset/legend/Pl_legend.svg',
    'Px': 'example_data/asset/legend/Px_legend.svg',
    'Qtz': 'example_data/asset/legend/Qtz_legend.svg'
}


# set the path of rose diagrams images
# Create a dictionary to store the paths of the images

rose_diagrams = {
    'Ap_UnW': 'example_data/asset/rose/PAL22_Ap_1.svg',
    'Ap_W': 'example_data/asset/rose/PAL22_Ap_2.svg',
    'Amph_UnW': 'example_data/asset/rose/PAL22_Amph_1.svg',
    'Amph_W': 'example_data/asset/rose/PAL22_Amph_2.svg',
    'Ep_UnW': 'example_data/asset/rose/PAL22_Ep_1.svg',
    'Ep_W': 'example_data/asset/rose/PAL22_Ep_2.svg',
    'Kfs_UnW' : 'example_data/asset/rose/PAL22_Kfs_1.svg',
    'Kfs_W' : 'example_data/asset/rose/PAL22_Kfs_2.svg',
    'Ol_UnW' : 'example_data/asset/rose/PAL22_Ol_1.svg',
    'Ol_W' : 'example_data/asset/rose/PAL22_Ol_2.svg',
    'Oth_UnW' : 'example_data/asset/rose/PAL22_Oth_1.svg',
    'Oth_W' : 'example_data/asset/rose/PAL22_Oth_2.svg',
    'Pl_UnW' : 'example_data/asset/rose/PAL22_Pl_1.svg',
    'Pl_W' : 'example_data/asset/rose/PAL22_Pl_2.svg',
    'Px_UnW' : 'example_data/asset/rose/PAL22_Px_1.svg',
    'Px_W' : 'example_data/asset/rose/PAL22_Px_2.svg',
    'Qtz_UnW' : 'example_data/asset/rose/PAL22_Qtz_1.svg',
    'Qtz_W' : 'example_data/asset/rose/PAL22_Qtz_2.svg',
}

blankDiagram = 'example_data/asset/rose/blank_diagram.svg'

#set the width of the map view after adding the legend icons
map_view_height='255px'

add_legend_and_rosediagrams('PAL22/index2.html','PAL22/index3.html', legend_icons, map_view_height, blankDiagram, rose_diagrams)

The folder "PAL22" contains the example_data results.

For a more comprension of editing process every step before generated a unique version of HTML file, the last file `PAL22/index3.html` is the final result.

You can delete other files (index.html, index2.html and openlayers.html) and rename index3.html as you like for put inside in a website or webpage (for example using \<iframe\> tag)

![video](https://i.imgur.com/NMS3e0R.mp4)